In [1]:
import plotly.graph_objects as go
import pandas as pd


In [2]:
%%time
pdf=pd.read_csv('C:/Data/short.tsv',sep='\t')
pdf.head()

Wall time: 43.8 s


,BUYER_CITY,BUYER_STATE,BUYER_ZIP,BUYER_COUNTY,DRUG_NAME,QUANTITY,TRANSACTION_DATE,Revised_Company_Name,dos_str,YEAR
0,GILBERT,AZ,85233,MARICOPA,HYDROCODONE,20,2010-03-23,Apotheca Inc.,5.0,2010
1,PHOENIX,AZ,85085,MARICOPA,HYDROCODONE,3,2010-09-15,Apotheca Inc.,5.0,2010
2,PHOENIX,AZ,85085,MARICOPA,HYDROCODONE,3,2010-12-09,Apotheca Inc.,7.5,2010
3,PULASKI,NY,13142,OSWEGO,HYDROCODONE,1,2010-12-02,"Allergan, Inc.",10.0,2010
4,PULASKI,NY,13142,OSWEGO,HYDROCODONE,5,2010-11-05,"Allergan, Inc.",10.0,2010


In [3]:
pdf['MG']=pdf['QUANTITY']*pdf['dos_str']
pdf.head()

,BUYER_CITY,BUYER_STATE,BUYER_ZIP,BUYER_COUNTY,DRUG_NAME,QUANTITY,TRANSACTION_DATE,Revised_Company_Name,dos_str,YEAR,MG
0,GILBERT,AZ,85233,MARICOPA,HYDROCODONE,20,2010-03-23,Apotheca Inc.,5.0,2010,100.0
1,PHOENIX,AZ,85085,MARICOPA,HYDROCODONE,3,2010-09-15,Apotheca Inc.,5.0,2010,15.0
2,PHOENIX,AZ,85085,MARICOPA,HYDROCODONE,3,2010-12-09,Apotheca Inc.,7.5,2010,22.5
3,PULASKI,NY,13142,OSWEGO,HYDROCODONE,1,2010-12-02,"Allergan, Inc.",10.0,2010,10.0
4,PULASKI,NY,13142,OSWEGO,HYDROCODONE,5,2010-11-05,"Allergan, Inc.",10.0,2010,50.0


In [4]:
%%time
# Total mg across each zip
df=pdf.groupby(['BUYER_ZIP','BUYER_STATE'])['MG'].sum().reset_index(name='TOTAL_MG')
#sort in descending order
df=df.sort_values('TOTAL_MG', ascending=False).reset_index(drop=True)
df.head()

Wall time: 4.94 s


,BUYER_ZIP,BUYER_STATE,TOTAL_MG
0,29405,SC,6.300415e+06
1,94550,CA,5.928099e+06
2,33614,FL,2.645837e+06
3,33613,FL,2.185500e+06
4,33607,FL,1.765315e+06


In [5]:
df2=pd.read_csv('population_density_zip.csv')
df2.head()

,ZIP_CODE,POPULATION,POPULATION_DENSITY_PER_SQ_MILE
0,1001,16769,1465.565461
1,1002,29049,527.751031
2,1003,10372,14587.904360
3,1005,5079,114.800416
4,1007,14649,278.270615


In [6]:
df3=pd.merge(df, df2, how='inner', left_on = 'BUYER_ZIP',right_on='ZIP_CODE')
df3.head()



,BUYER_ZIP,BUYER_STATE,TOTAL_MG,ZIP_CODE,POPULATION,POPULATION_DENSITY_PER_SQ_MILE
0,29405,SC,6.300415e+06,29405,26328,1770.306616
1,94550,CA,5.928099e+06,94550,46896,201.408693
2,33614,FL,2.645837e+06,33614,46449,4750.357946
3,33613,FL,2.185500e+06,33613,31990,4354.750885
4,33607,FL,1.765315e+06,33607,23541,2606.688074


In [7]:
df4=df3.groupby(['BUYER_STATE'])['TOTAL_MG','POPULATION'].sum().reset_index()
df4.head()

,BUYER_STATE,TOTAL_MG,POPULATION
0,AK,1.669891e+06,572378
1,AL,1.566770e+07,4178099
2,AR,9.396343e+06,2471502
3,AZ,2.150076e+07,5917270
4,CA,8.936904e+07,35999587


In [17]:
fig = go.Figure(data=go.Choropleth(
    locations=df4['BUYER_STATE'], # Spatial coordinates
    z = (df4['TOTAL_MG']/df4['POPULATION']).astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Blues',
    colorbar_title = "Qty",
))

fig.update_layout(
    title_text = '2010 Total mg of Opioids vs State, normalized by Population',
    geo_scope='usa', # limite map scope to USA
)

fig.hide()

AttributeError: 'Figure' object has no attribute 'hide'

In [18]:
fig.show()